This code illustrates how to use the aggregation code and extrapolate some informations from the aggregates at each aggregation step

I am using the "*generate_rimed_aggregate*" function again because I think this is the one we used most of the times.

**Important** is to pass the argument "*iter=True*" such that we get an iterator and not just the final product!

At each aggregation step we have a list of aggregates which gets shorter by 1 element due to an aggregation event. The objective is to catch that aggregation event and print some properties of the particles. By default, only the particles that collide are modified (rotated and aggregated), the others are left untouched, so, to avoid redundancy, we save only the "new" particles

Some properties of the new particles are sort of straightforward, other are less. For example, the orientation of the colliding particles is lost after collision. To retrieve it I split again the aggregate after collision in its constituents taking advantage of the id_tree attribute. With such aggregates isolated I can calculate their orientations separately and store, for example, the polar and azimuth angle of the shortest principal axis of inertia. This is relevant since, by default, at each aggregation step, the code aligns this axis to be vertical and then it picks a completely random azimuth while the polar angle comes from a gaussian distribution with std=40deg (this default behaviour is hard-coded, but there are ways around it)

In [1]:
from aggregation import riming_runs
from aggregation.aggregate import Aggregate
import pandas as pd
import numpy as np
from collections.abc import Iterable


# Set the experiment up
N = 5
grid_res = 40e-6 # meters
voxel_volume = grid_res**3 # m**3
ice_density = 917.0 # kg m**-3
voxel_mass = voxel_volume*ice_density

gen = riming_runs.gen_monomer(psd="monodisperse", size=300e-6, min_size=100e-6, max_size=3000e-6, mono_type="dendrite", rimed=True, grid_res=grid_res)
agg_iter = riming_runs.generate_rimed_aggregate(gen, N=N, align=True, riming_lwp=0.0, riming_mode="subsequent", lwp_div=100, iter=True)


def hash_aggregate(agg): # for now I use just the string representation as index, maybe in the future substitute with hash(str(agg.id_tree)) to have a unique integer
    return str(agg.id_tree)


def gen_flatten(xs): # this I use to unravel the id_tree
    for x in xs:
        if isinstance(x, Iterable):
            yield from flatten(x)
        else:
            yield x

            
def flatten(xs):
    gen = gen_flatten(xs)
    if isinstance(xs, Iterable):
        return list(gen)
    else:
        return [xs]
    

class reloadAggregate(Aggregate):
    def __init__(self, X, resolution, ident, id_tree, mono_num):
        self.X = X
        self.grid_res = resolution
        self.ident = ident
        self.id_tree = id_tree
        self.monomer_number = mono_num
        self.update_extent() # this is important!


def split_aggregate(agg):
    """
    This function take an aggregate and splits it into its previous (before collision) situation by looking at the id_tree attribute
    """
    tree0 = agg.id_tree[0]
    tree1 = agg.id_tree[1]
    list0 = flatten(tree0)
    list1 = flatten(tree1)
    mask0 = [i in list0 for i in agg.ident]
    mask1 = [i in list1 for i in agg.ident] # it is probably faster to vectorize not(mask0)
    ident0 = agg.ident[mask0]
    X0 = agg.X[mask0, :]
    ident1 = agg.ident[mask1]
    X1 = agg.X[mask1, :]
    agg0 = reloadAggregate(X0, grid_res, ident0, tree0, len(np.unique(ident0)))
    agg1 = reloadAggregate(X1, grid_res, ident1, tree1, len(np.unique(ident1)))
    return agg0, agg1


# Prepare an empty Dataframe where we will store the needed informations and the function that fills it
cols = ['generation', 'monomer_number', 'mass', 'aspect_ratio', 'agg0_polar_axz', 'agg0_azim_axz', 'agg1_polar_axz', 'agg1_azim_axz'] # make a list of properties
df = pd.DataFrame(columns=cols)

def record_aggregate(agg, iteration):
    idx = hash_aggregate(agg)
    if idx not in df.index: # add only newly generated shapes
        
        # make an empty entry
        row = pd.DataFrame(index=[idx], columns=cols)
        
        # Fill the attributes of the entry with some basic stuff
        row['generation'] = int(iteration)
        row['monomer_number'] = i.monomer_number
        row['mass'] = i.X.shape[0]*voxel_mass
        row['aspect_ratio'] = agg.aspect_ratio()
        
        # Something more elaborate (orientation of aggregates prior collision)
        if i.monomer_number-1: # you can only split aggregates
            agg0, agg1 = split_aggregate(agg)
            ax0 = agg0.principal_axes()
            ax0z = ax0[:, 2] # the shortest principal axis is the last column
            r0 = np.sqrt((ax0z*ax0z).sum())
            ax1 = agg1.principal_axes()
            ax1z = ax1[:, 2]
            r1 = np.sqrt((ax1z*ax1z).sum())
            
            row['agg0_polar_axz'] = np.arccos(ax0z[2]/r0)
            row['agg0_azim_axz'] = np.arctan2(ax0z[1], ax0z[0])
            row['agg1_polar_axz'] = np.arccos(ax1z[2]/r1)
            row['agg1_azim_axz'] = np.arctan2(ax1z[1], ax1z[0])
        
        df.loc[idx] = row.loc[idx]



In [2]:
# Print the number of monomers in each aggregate at every aggregation step
for iteration, aggs in enumerate(agg_iter):
    for i in aggs:
        record_aggregate(i, iteration)

In [3]:
# Let's see some results
df

,generation,monomer_number,mass,aspect_ratio,agg0_polar_axz,agg0_azim_axz,agg1_polar_axz,agg1_azim_axz
0,0,1,8.803200e-10,0.206725,NaN,NaN,NaN,NaN
1,0,1,8.803200e-10,0.206725,NaN,NaN,NaN,NaN
2,0,1,8.803200e-10,0.206725,NaN,NaN,NaN,NaN
3,0,1,8.803200e-10,0.206725,NaN,NaN,NaN,NaN
4,0,1,8.803200e-10,0.206725,NaN,NaN,NaN,NaN
"[3, 4]",1.0,2.0,1.760640e-09,0.318316,1.462118,0.937516,1.617450,1.198833
"[[3, 4], 2]",2.0,3.0,2.640960e-09,0.302171,0.141790,0.362885,1.898394,0.571979
"[[[3, 4], 2], 0]",3.0,4.0,3.521280e-09,0.326943,0.306180,-2.626215,0.961308,0.845618
"[[[[3, 4], 2], 0], 1]",4.0,5.0,4.401600e-09,0.416659,0.644357,0.209604,1.695962,-0.263659


As you can see. In the table you find the properties of all the initial monomers which are marked as generation 0. All subsequent particles are generated by an aggregation event which is identified by the progressive generation integer.